In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#import Levenshtein
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize
from nltk.stem import SnowballStemmer
from nltk.metrics import jaccard_distance


In [22]:
stemmer = SnowballStemmer("english")

In [23]:
#from gensim import corpora, models

In [24]:
from nltk.corpus import stopwords

In [25]:
#from gensim.models import Word2Vec
#from sklearn.metrics.pairwise import euclidean_distances
#from scipy.spatial.distance import euclidean

In [26]:
# Make sure to download NLTK resources
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('maxent_ne_chunker')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('words')
#nltk.download('stopwords')

In [27]:
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [28]:
from nltk.translate.meteor_score import meteor_score

In [41]:
#data = pd.read_csv(r'C:\Users\Abdullah\Text_Evaluation_Metrics\dataset\test.csv',sep = ',')
#data = pd.read_csv(r'C:\Users\Abdullah\Text_Evaluation_Metrics\dataset\billsum\ustest_processed.csv',sep = ',')
#data = pd.read_csv(r'C:\Users\Abdullah\Text_Evaluation_Metrics\dataset\bbc_news\BBCarticles_csv.csv',sep = ',',encoding='latin-1')
#data = pd.read_csv(r'C:\Users\Abdullah\Text_Evaluation_Metrics\dataset\Samsum\samsum-test.csv',sep = ',')
#data = pd.read_csv(r'C:\Users\Abdullah\Text_Evaluation_Metrics\dataset\dialog_sum\CSV\test.csv',sep = ',')
### machine learning model ######
data = pd.read_csv(r'C:\Users\Abdullah\Text_Evaluation_Metrics\dataset\t5_bbc\t5_bbc_test.csv',sep = ',',encoding='latin-1')

In [42]:
data.head()

,Unnamed: 0,Article,original_Summary,Predicted_Summary
0,0,Ad sales boost Time Warner profit\n\nQuarterly...,TimeWarner said fourth quarter sales rose 2% t...,revenues rose 6.4% to $42.09bn. TimeWarner pos...
1,1,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...,"at $1.2871 against the euro, from $1.2974 on T..."
2,2,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...,owners of embattled Russian oil giant to ask b...
3,3,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ...",it made a pre-tax profit of Â£75m ($141m) comp...
4,4,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...,Ricard. Allied Domecq shares in London rose 4%...


In [43]:
#new_column_names = {'clean_text': 'article', 'summary': 'highlights'} # billsum
#new_column_names = {'Text': 'article', 'Summary': 'highlights'} # bbc
#new_column_names = {'dialogue': 'article', 'summary': 'highlights'} # samsum
#new_column_names = {'dialogue': 'article', 'summary': 'highlights'} # dialog sum
#new_column_names = {'inputs': 'article', 'inferences': 'highlights'} # dialog sum
### machine learning model
new_column_names = {'Article': 'article', 'Predicted_Summary': 'highlights'} # t5 bbc 
data.rename(columns=new_column_names, inplace=True)

In [44]:
#!pip install python-Levenshtein

In [45]:
data.head()

,Unnamed: 0,article,original_Summary,highlights
0,0,Ad sales boost Time Warner profit\n\nQuarterly...,TimeWarner said fourth quarter sales rose 2% t...,revenues rose 6.4% to $42.09bn. TimeWarner pos...
1,1,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...,"at $1.2871 against the euro, from $1.2974 on T..."
2,2,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...,owners of embattled Russian oil giant to ask b...
3,3,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ...",it made a pre-tax profit of Â£75m ($141m) comp...
4,4,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...,Ricard. Allied Domecq shares in London rose 4%...


In [46]:

#article_values = data['article'].tolist()
article_values = data['original_Summary'].tolist()
summary_values = data['highlights'].tolist()

In [47]:
# Print the extracted lists
print("Values from column 'article':", len(article_values))
print("Values from column 'highlights':", len(summary_values))

Values from column 'article': 2225
Values from column 'highlights': 2225


In [48]:
# Define a function for data cleaning
def clean_text(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Remove stop words
    words = [word for word in words if word.lower() not in stopwords.words("english")]
    
    # Apply stemming
    words = [stemmer.stem(word) for word in words]
    
    # Rejoin the cleaned words into a single string
    cleaned_text = " ".join(words)
    cleaned_text = cleaned_text.replace('\n', ' ').replace('\r', ' ')
    
    return cleaned_text

## Parameter and Methods 

### 1.Coherence and Cohesion : LSA+Cosine Similarity 
### 2.Relevance : METEOR 
### 3.Redundancy : Cosine Similarity (only on summary text)
### 4.Bias Avoidance : NER + Jaccard Similarity 
### 5.Conciseness : LDA + Jaccard Similarity 

In [49]:
def calculate_score(original_content, summary):
    # Apply data cleaning to the original content and summary
    cleaned_original_content = clean_text(original_content)
    cleaned_summary = clean_text(summary)
    #cleaned_human_summary = clean_text(summary)
    #print('cleaned_original_content',cleaned_original_content)
    #print('\n')
    #print('cleaned_summary',cleaned_summary)
    #print('\n')
    
    # Factor weights (can be adjusted based on importance)
    factor_weights = {
        'Semantic Similarity': 0.25,
        'Relevance': 0.25,
        #'Coherence and Cohesion': 0.30,
        'Redundancy': 0.25,
        'Bias Avoidance': 0.25
        #'Conciseness': 0.25
    }
    # Factor scores (on a scale from 0 to 100)
    factor_scores = {}
    
    # Factor: Semantic Similarity
    # (Use Latent Semantic Analysis (LSA) + CS)
    
    lsa_score = semantic_similarity_score(original_content, summary)
    factor_scores['Semantic Similarity'] = 100 * lsa_score
    
    # Factor: Relevance
    #Method used : METEOR
    relevance_score = relevance_similarity(original_content, summary)
    factor_scores['Relevance'] = 100 * relevance_score
    
    
    
    # Factor: Redundancy
    # Calculate redundancy using cosine similarity between summary sentences (CS)
    redundancy_score = redundancy_analysis(summary)
    factor_scores['Redundancy'] = 100 * redundancy_score
     
    
    # Factor: Bias Avoidance
    # (Use Named Entity Recognition + JS)
    bias_avoidance_score = bias_avoidance_analysis(original_content, summary)
    factor_scores['Bias Avoidance'] = 100 * bias_avoidance_score
    
    # Calculate overall score as weighted sum of factor scores
    overall_score = sum(factor_scores[factor] * factor_weights[factor] for factor in factor_weights)/100
    return overall_score



In [50]:
def calculate_average_score(original_contents, summaries):
    total_score = 0
    for original_content, summary in zip(original_contents, summaries):
        score = calculate_score(original_content, summary)
        ##print('Score: ',score)
        total_score += score
    average_score = total_score / len(original_contents)
    return average_score

In [51]:

def cosine_similarity_score_for_redundancy(vector1, vector2):
    similarity = cosine_similarity(vector1, vector2)[0][0]
    return similarity

# Perform Latent Semantic Analysis (LSA) on the original content and summary
# this function calculates the cosine similarity between two vectors. The vectors vector1 and vector2 are reshaped to ensure
#that they have the same shape before computing the cosine similarity.    
def calculate_cosine_similarity(vector1, vector2):
    similarity = cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))
    return similarity[0][0]

def semantic_similarity_score(original_content, summary):
    # Perform Latent Semantic Analysis (LSA) on the original content and summary
    documents = [original_content, summary]

    # Create a document-term matrix using TfidfVectorizer
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(documents)

    # Apply LSA for dimensionality reduction
    num_topics = 2  # Number of topics (latent dimensions)
    lsa = TruncatedSVD(n_components=num_topics)
    X_lsa = lsa.fit_transform(X)

    # Calculate cosine similarity between the original text and its summary
    cosine_similarity_score = calculate_cosine_similarity(X_lsa[0], X_lsa[1])

    ###print("Cosine Similarity Score (LSA) for Coherence and Cohesion:", cosine_similarity_score)
    lsa_sim = cosine_similarity_score
    return lsa_sim


def relevance_similarity(original_content, summary):
    # Implement METEOR to evaluate the relevance of the summary to the original content
    ### higher score is better ####
    main_text =  word_tokenize(original_content)
    summary =  word_tokenize(summary)

    # Calculate METEOR score
    meteor = meteor_score([main_text], summary)
    ###print("relevance_similarity (meteor):", meteor)
    return meteor

def redundancy_analysis(summary):
    sentences = sent_tokenize(summary)
    # Handle the case of a single sentence summary
    if len(sentences) == 1:
        return 1.0
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(sentences)
    
    similarity_threshold = 0.5
    redundant_count = 0
    
    for i in range(len(sentences)):
        ##print('base sentences: ', sentences[i])
        for j in range(i + 1, len(sentences)):
            ##print('com sentences: ', sentences[j])
            similarity_score = cosine_similarity_score_for_redundancy(vectors[i], vectors[j])
            ##print('similarity_score:',similarity_score)
            if similarity_score < similarity_threshold:
                redundant_count += 1
        ##print('......................................')
    # Calculate redundancy score as a ratio of redundant pairs to total pairs
    total_pairs = (len(sentences) * (len(sentences) - 1)) / 2
    ##print('redundant_count: ',redundant_count)
    ##print('total_pairs: ',total_pairs)
    ##redundancy_score = 1 - (redundant_count / total_pairs)
    redundancy_score = (redundant_count / total_pairs)
    ###print("redundancy_analysis (cosine similarity)", redundancy_score)
    return redundancy_score
    

def bias_avoidance_analysis(original_content, summary):
    # Use Named Entity Recognition (NER) to identify named entities in the summary
    ### higher score is better ###
    original_entities = extract_named_entities(original_content)
    ##print('-------------------------------------------------------')
    ##print('original_entities: ',original_entities)
    summary_entities = extract_named_entities(summary)
    ##print('-------------------------------------------------------')
    ##print('summary_entities: ',summary_entities)
    
    # Calculate the Jaccard similarity of named entities to measure bias avoidance
    jaccard_score = calculate_jaccard_similarity(original_entities, summary_entities)
    ###print("bias_avoidance_analysis (NER)", jaccard_score)
    return jaccard_score

def extract_named_entities(text):
    entities = []
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        pos_tags = nltk.pos_tag(words)
        chunked = ne_chunk(pos_tags)
        for subtree in chunked:
            if isinstance(subtree, nltk.Tree):
                entity = " ".join([word for word, tag in subtree.leaves()])
                entities.append(entity)
    return set(entities)

def calculate_jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    jaccard_similarity = intersection / union if union > 0 else 0
    return jaccard_similarity


In [13]:
# Example usage:

In [14]:
#original_content = 'A big brown fox jump over the lazy dog. The dog is small in size'
#summary = 'A large brown fox leaped across the lethargic dog. The canine is of a diminutive stature.'

#original_content = 'Munich is one of the biggest city in Germany. It has a football club '
#summary = 'Bayern Munich is a socker club in Germany. It is a famous club'


#######abstractive summary #############
original_content ='''
The concept of artificial intelligence (AI) has evolved dramatically over the past few decades. AI, initially thought to be confined to science fiction, is now an integral part of our daily lives. This transformation is largely due to the exponential growth in computing power and the development of sophisticated algorithms.

In the early days of AI, the focus was on rule-based systems, where programmers explicitly defined the rules for the computer to follow. These systems had limited capabilities and struggled with complex, real-world problems. However, as computing power increased, machine learning techniques emerged.

Machine learning allows AI systems to learn from data and make predictions or decisions without being explicitly programmed. This shift has led to remarkable advances in areas such as natural language processing, computer vision, and recommendation systems. AI-powered applications are now commonplace, from virtual personal assistants like Siri to autonomous vehicles.

One of the most exciting AI developments is deep learning, a subset of machine learning that involves neural networks inspired by the human brain. Deep learning models, particularly convolutional neural networks (CNNs) and recurrent neural networks (RNNs), have achieved groundbreaking results in image and speech recognition, language translation, and more. They can process massive datasets and extract intricate patterns, leading to remarkable accuracy.

The impact of AI is felt across industries, including healthcare, finance, and manufacturing. In healthcare, AI is used for disease diagnosis, drug discovery, and personalized treatment plans. Financial institutions rely on AI for fraud detection, algorithmic trading, and customer service chatbots. Manufacturers employ AI-powered robotics and automation for increased efficiency and quality control.

Despite its transformative potential, AI also raises ethical and societal concerns. The collection and use of personal data, algorithmic biases, and the potential for job displacement are among the challenges that must be addressed. Regulatory frameworks and ethical guidelines are being developed to ensure responsible AI development and deployment.

Looking ahead, the future of AI holds promise and uncertainty. As AI systems become more capable and autonomous, questions about their decision-making processes and accountability become more pressing. Continued research, collaboration, and public discourse are essential to navigate the complex landscape of AI and ensure that it benefits society as a whole.
'''

summary = '''
Artificial intelligence (AI) has undergone a profound evolution, moving from science fiction to everyday reality. This transformation is driven by increased computing power and the rise of machine learning, allowing AI systems to learn and make decisions autonomously. Deep learning, inspired by the human brain, has pushed the boundaries of AI, enabling breakthroughs in areas like language processing and image recognition.

AI's impact spans industries, from healthcare and finance to manufacturing, revolutionizing processes and decision-making. However, ethical concerns, data privacy, and potential job displacement pose challenges. Regulatory efforts aim to ensure responsible AI development.

The future of AI holds great promise but demands careful navigation of ethical and societal issues, making ongoing research and public dialogue crucial for harnessing AI's potential for the benefit of society.

'''

In [15]:
score = calculate_score(original_content, summary)
print(f"Overall Score: {score:.4f}")

Overall Score: 0.5496


In [52]:
###### summary does not match with context ###
original_content = '''
In the heart of the bustling city, there stood a magnificent cathedral, a testament to centuries of artistry and devotion. Its towering spires reached for the heavens, and its stained glass windows bathed the interior in a kaleidoscope of colors. Pilgrims and tourists alike marveled at its intricate architecture, a blend of Gothic and Renaissance styles.

Inside, the cathedral was a sanctuary of serenity. Sunlight streamed through the windows, casting ethereal patterns on the stone floor. The echoes of whispered prayers filled the air, and the scent of incense lingered, a fragrant offering to the divine.

Generations had contributed to this sacred place, from the master craftsmen who sculpted its façade to the faithful who tended its gardens. It was a living legacy, a symbol of both faith and human ingenuity.
'''

summary = '''
Amidst the urban chaos, a hidden garden thrived, a living canvas of vibrant life. Its lush foliage sprawled, forming a lush tapestry of greens and reds, a testament to nature's artistry. Birds flitted in and out, their songs a colorful symphony.

Within this haven, tranquility reigned. Sunlight dappled the earth, casting playful shadows. Laughter and music filled the air, a celebration of life's beauty, while the scent of blooming flowers enveloped all.

Generations of caretakers nurtured this oasis, from the green-thumbed gardeners who tended its blooms to the children who played among its wonders. It was a living testament, a symbol of nature's resilience and human stewardship
'''

In [53]:
score = calculate_score(original_content, summary)
print(f"Overall Score: {score:.4f}")

-------------------------------------------------------
original_entities:  {'Renaissance', 'Gothic', 'Sunlight', 'Inside'}
-------------------------------------------------------
summary_entities:  {'Sunlight'}
Overall Score: 0.5396


In [52]:
#score = calculate_score(original_content, summary)
#print(f"Overall Score: {score:.2f}")
average_score = calculate_average_score(article_values, summary_values)
print(f"Average Score: {average_score:.4f}")

Average Score: 0.4729


In [27]:
###########Average Score: 0.4896 (after cleaning)
#0.4783 (3331)
# 0.4276 (all same)

#Average Score: 0.4824 (all same billsum)
#Average Score: 0.6539 (all same bbc)
#Average Score: 0.5108 (all same samsum)
#Average Score: 0.4918 (all same dialog sum)

In [16]:
#### Summary 1 (Real Abstractive Summary):
original_content ='''
Artificial intelligence (AI) has undergone a profound evolution, moving from science fiction to everyday reality. 
This transformation is driven by increased computing power and the rise of machine learning, allowing AI systems to 
learn and make decisions autonomously. Deep learning, inspired by the human brain, has pushed the boundaries of AI, 
enabling breakthroughs in areas like language processing and image recognition. AI's impact spans industries, 
from healthcare and finance to manufacturing, revolutionizing processes and decision-making. However, ethical concerns, 
data privacy, and potential job displacement pose challenges. Regulatory efforts aim to ensure responsible AI development. 
The future of AI holds great promise but demands careful navigation of ethical and societal issues, making ongoing research 
and public dialogue crucial for harnessing AI's potential for the benefit of society.
'''

summary = '''
The evolution of artificial intelligence (AI) from science fiction to everyday reality has been driven by increased computing 
power, machine learning, and deep learning. AI has revolutionized multiple industries but raised concerns about ethics, data 
privacy, and job displacement. Ethical regulation is crucial for responsible AI development.
'''
score = calculate_score(original_content, summary)
print(f"Overall Score: {score:.4f}")

Overall Score: 0.4667


In [17]:
#### Summary 2 (Non-Matching Summary): 
original_content ='''
Artificial intelligence (AI) has undergone a profound evolution, moving from science fiction to everyday reality. 
This transformation is driven by increased computing power and the rise of machine learning, allowing AI systems to 
learn and make decisions autonomously. Deep learning, inspired by the human brain, has pushed the boundaries of AI, 
enabling breakthroughs in areas like language processing and image recognition. AI's impact spans industries, 
from healthcare and finance to manufacturing, revolutionizing processes and decision-making. However, ethical concerns, 
data privacy, and potential job displacement pose challenges. Regulatory efforts aim to ensure responsible AI development. 
The future of AI holds great promise but demands careful navigation of ethical and societal issues, making ongoing research 
and public dialogue crucial for harnessing AI's potential for the benefit of society.
'''

summary = '''
The incredible rise of AI, like a phoenix from the ashes, has altered our world. As AI blossoms, 
we must cherish the seeds of responsibility, nurture them into robust trees of ethical practice, 
and keep them vibrant through a symphony of public discourse. In the end, AI is a canvas where the brushstrokes of 
society paint the future.
'''
score = calculate_score(original_content, summary)
print(f"Overall Score: {score:.4f}")

Overall Score: 0.4716


In [18]:
#original_content ='There is a brown fox. It is really quick. It jumped over the lazy dog. The dog is small in size.'
original_content ='''
In a serene landscape, a quick brown fox adds a burst of energy to the scene with its remarkable agility. Its luscious brown fur seamlessly blends with the natural surroundings as it gracefully leaps over a small, slumbering dog. Despite its compact size, the lazy dog remains undisturbed, embodying the essence of canine relaxation. This impromptu performance becomes a picturesque moment, where the fox's swift movements during the jump inject a burst of liveliness into the otherwise tranquil tableau of the lazy dog's repose.
'''
summary =  'The quick brown fox jumped over the lazy fox'
score = calculate_score(original_content, summary)
print(f"Overall Score: {score:.4f}")

Overall Score: 0.3756


In [19]:
#summary ='There is a brown fox. It is really quick. It jumped over the lazy dog. The dog is small in size.'
original_content ='''
In a serene landscape, a quick brown fox adds a burst of energy to the scene with its remarkable agility. Its luscious brown fur seamlessly blends with the natural surroundings as it gracefully leaps over a small, slumbering dog. Despite its compact size, the lazy dog remains undisturbed, embodying the essence of canine relaxation. This impromptu performance becomes a picturesque moment, where the fox's swift movements during the jump inject a burst of liveliness into the otherwise tranquil tableau of the lazy dog's repose.
'''
#original_content =  'The fast wood-coloured fox hopped over the lethargic dog'
summary  = 'A nimble, wood-coloured fox swiftly leaped above the lethargic and petite dog.'
score = calculate_score(original_content, summary)
print(f"Overall Score: {score:.4f}")

Overall Score: 0.3046
